In [14]:
#Step 13 calculates the demographic information of patients in the MGB and BIDMC sampling cohorts. 

In [15]:
#Import all of the necessary modules. 
import pandas as pd
import numpy as np 
from tqdm import tqdm
from thunderpack import ThunderReader  


In [16]:
#Create the demographics csvs for each hospital

#Load the CSV files into DataFrames
bidmc = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/BIDMC_Complete_Notes.csv")
mgb = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/MGB_Complete_Notes.csv")

#Concatenate the DataFrames
concatenated_df = pd.concat([bidmc, mgb], ignore_index=True)

#Save the concatenated DataFrame to a new CSV file
concatenated_df.to_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/MGB_BIDMC_Complete_Notes.csv", index=False)
notes = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/MGB_BIDMC_Complete_Notes.csv")

#Extract the relevant columns.
columns_of_interest = ['BDSPPatientID', 'ContactDate', 'Site']
notes_filtered = notes[columns_of_interest]

#Separate the DataFrame based on hospitals in the "Site" column.
notes_mgb = notes_filtered[notes_filtered['Site'] == 'MGB']
notes_bidmc = notes_filtered[notes_filtered['Site'] == 'BIDMC']

#Save the separated DataFrames to new CSV files.
notes_mgb.to_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/ID_and_Notes_mgb.csv", index=False)
notes_bidmc.to_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/ID_and_Notes_bidmc.csv", index=False)
print("CSV files have been created successfully.")

CSV files have been created successfully.


In [17]:
#Verifying the data is still aligned

#Load the original CSV file.
original_notes = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/MGB_BIDMC_Complete_Notes.csv")

#Load the new CSV files.
notes_mgb = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/ID_and_Notes_mgb.csv")
notes_bidmc = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/ID_and_Notes_bidmc.csv")

#Extract and align columns.
original_notes_filtered = original_notes[['BDSPPatientID', 'ContactDate', 'Site']]
notes_mgb_filtered = notes_mgb[['BDSPPatientID', 'ContactDate', 'Site']]
notes_bidmc_filtered = notes_bidmc[['BDSPPatientID', 'ContactDate', 'Site']]

#Verify if the new DataFrames align with the original DataFrame.
def verify_alignment(original_df, new_df, label):
    original_sorted = original_df.sort_values(by=['ContactDate', 'Site']).reset_index(drop=True)
    new_sorted = new_df.sort_values(by=['ContactDate', 'Site']).reset_index(drop=True)
    
    if original_sorted.equals(new_sorted):
        print(f"{label} alignment correct.")
    else:
        print(f"{label} alignment incorrect.")
        #Find rows that are different.
        original_diff = original_sorted.compare(new_sorted, result_names=("Original", "New"))
        print("Differences:")
        print(original_diff)
        #Rows in original but not in new.
        missing_in_new = original_sorted[~original_sorted.isin(new_sorted).all(axis=1)]
        print("Rows in original but not in new:")
        print(missing_in_new)
        #Rows in new but not in original.
        missing_in_original = new_sorted[~new_sorted.isin(original_sorted).all(axis=1)]
        print("Rows in new but not in original:")
        print(missing_in_original)

#Separate the original DataFrame for MGB and BIDMC to compare.
original_notes_mgb = original_notes_filtered[original_notes_filtered['Site'] == 'MGB']
original_notes_bidmc = original_notes_filtered[original_notes_filtered['Site'] == 'BIDMC']

#Verify the alignment. 
verify_alignment(original_notes_mgb, notes_mgb_filtered, "MGB")
verify_alignment(original_notes_bidmc, notes_bidmc_filtered, "BIDMC")


MGB alignment correct.
BIDMC alignment correct.


In [18]:
#Read the Thunderpack file for BIDMC, pull out the relevant information, and calculate the age of patients. 
reader = ThunderReader('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/thunderpack_demographics_nax_1m_BIDMC')

#Initialize a list to hold the DataFrames.
dfs = []

#Loop over all partitions to concatenate DataFrames.
for key in reader.keys():
    df = reader[key]
    dfs.append(df)

#Concatenate all DataFrames into one.
df_demograph_BI = pd.concat(dfs, axis=0, ignore_index=True)

#Keep the relevant columns and remove duplicates. 
df_demograph_BI = df_demograph_BI[['BDSPPatientID', 'PatientRace', 'EthnicGroupDSC', 'SexDSC']].drop_duplicates()

#Load the notes_bidmc DataFrame. 
notes_bidmc = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/ID_and_Notes_bidmc.csv")

#Extract the relevant columns. 
notes_bidmc = notes_bidmc[['BDSPPatientID', 'ContactDate', 'Site']]

#Verify that all IDs in notes_bidmc are present in df_demograph_BI. 
missing_ids = notes_bidmc[~notes_bidmc['BDSPPatientID'].isin(df_demograph_BI['BDSPPatientID'])]
if not missing_ids.empty:
    print("Warning: Some BDSPPatientID values in notes_bidmc do not have corresponding entries in df_demograph_BI:")
    print(missing_ids[['BDSPPatientID']])

#Filter df_demograph_BI to include only those IDs present in notes_bidmc. 
filtered_demograph = df_demograph_BI[df_demograph_BI['BDSPPatientID'].isin(notes_bidmc['BDSPPatientID'])]

#Merge the filtered demographic DataFrame with notes_bidmc.
merged_df = pd.merge(notes_bidmc, filtered_demograph, on='BDSPPatientID', how='left')

#Load the birth dates CSV file.
birth_dates = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/bidmc_patient_demographics_birth_and_death_dates.csv')

#Extract the relevant columns. 
birth_dates = birth_dates[['BDSPPatientID', 'DateOfBirth']]

#Verify that all IDs in birth_dates are present in merged_df. 
missing_birth_dates = birth_dates[~birth_dates['BDSPPatientID'].isin(merged_df['BDSPPatientID'])]
if not missing_birth_dates.empty:
    print("Warning: Some BDSPPatientID values in birth_dates do not have corresponding entries in merged_df:")
    print(missing_birth_dates[['BDSPPatientID']])

#Merge the birth dates DataFrame with merged_df. 
final_merged_df = pd.merge(merged_df, birth_dates, on='BDSPPatientID', how='left')

#Check for BDSPPatientID with missing DateOfBirth after the merge. 
missing_date_of_birth = final_merged_df[final_merged_df['DateOfBirth'].isna()]
if not missing_date_of_birth.empty:
    print("Warning: Some BDSPPatientID values in final_merged_df are missing DateOfBirth:")
    print(missing_date_of_birth[['BDSPPatientID']])

#Convert ContactDate and DateOfBirth to datetime objects. 
final_merged_df['ContactDate'] = pd.to_datetime(final_merged_df['ContactDate'], errors='coerce')
final_merged_df['DateOfBirth'] = pd.to_datetime(final_merged_df['DateOfBirth'], errors='coerce')

#Calculate Age as the difference between ContactDate and DateOfBirth.
final_merged_df['Age'] = (final_merged_df['ContactDate'] - final_merged_df['DateOfBirth']).dt.days // 365

#Save the final merged DataFrame with age calculation to a new CSV file. 
final_merged_df.to_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/notes_bidmc_with_demographics_and_birth_dates_and_age.csv", index=False)
print("Final merged DataFrame with demographics, birth dates, and age calculation saved successfully.")


         BDSPPatientID
0            150000000
1            150059448
2            150000001
3            150060112
4            150060113
...                ...
1400253      151400253
1400254      151400254
1400255      151400255
1400256      151400256
1400257      151400257

[1398758 rows x 1 columns]
Final merged DataFrame with demographics, birth dates, and age calculation saved successfully.


In [19]:
#Printing values that are missing to recover if information is made available later. 
#Read the CSV file into a DataFrame. 
df = pd.read_csv('/home/gregory178/Desktop/NAX project/NAX_SDH/notes_bidmc_with_demographics_and_birth_dates_and_age.csv')

#Identify the rows with missing values.
missing_values = df.isna().any(axis=1)

#Extract BDSPPatientID where there are missing values. 
patient_ids_with_missing_values = df.loc[missing_values, 'BDSPPatientID']

#Convert to a list and count the number of patient IDs. 
patient_ids_list = patient_ids_with_missing_values.tolist()
count_patient_ids = len(patient_ids_list)

#Print the BDSPPatientID values and the count. 
print("Patient IDs with missing values:", patient_ids_list)
print("Total count of patient IDs with missing values:", count_patient_ids)


Patient IDs with missing values: [150011240, 151156613, 151133857, 150676236, 150010619, 150722344, 151126349, 151153026, 151119406, 151065320, 151124644, 151121293, 151120647, 150751070, 150011561, 150654988, 150670494, 150921653, 150011241, 151152749, 150009013, 151150423, 150010846, 151062252, 151130741, 150099308, 150005606, 151035318, 151036497, 151156430, 151154144, 151131036, 151146593, 151080094, 151136386, 151130102, 150595967, 150816792, 151133268, 151116152, 150785930, 150011218, 150008749, 150981926, 151101091, 150754899, 150393824, 150946700, 150947713, 151125701, 150534531, 150772328, 150011731, 150595603, 151123111, 151155621, 150570234, 150093117, 150607610, 151152805, 151111413, 150138965, 151131568, 150977822, 150650421, 151153673, 151128399, 151151212, 151082531, 151016742, 150863809, 151149625, 151135013]
Total count of patient IDs with missing values: 73


In [20]:
#Read the Thunderpack file for MGB, pull out the relevant information, and calculate the age of patients. 
#Read the Thunderpack file for MGB.
reader = ThunderReader('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/MGB/thunderpack_demographics_MGB')

#Initialize a list to hold the DataFrames.
dfs = []

#Loop over all partitions to concatenate DataFrames.
for key in reader.keys():
    df = reader[key]
    if isinstance(df, pd.DataFrame):
        dfs.append(df)
    else:
        print(f"Warning: Object for key '{key}' is not a DataFrame, it is a {type(df)}.")

#Check if dfs is empty.
if not dfs:
    raise ValueError("No DataFrames found. Please check the ThunderReader output.")

#Concatenate all of the DataFrames into one. 
df_demograph_MGB = pd.concat(dfs, axis=0, ignore_index=True)

#Keep the relevant columns and remove duplicates. 
df_demograph_MGB = df_demograph_MGB[['BDSPPatientID', 'PatientRace', 'EthnicGroupDSC', 'SexDSC', 'DateOfBirth']].drop_duplicates()

#Load the notes_mgb DataFrame. 
notes_mgb = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/ID_and_Notes_mgb.csv")  

#Extract the relevant columns. 
notes_mgb = notes_mgb[['BDSPPatientID', 'ContactDate', 'Site']]

#Verify that all the IDs in notes_mgb are present in df_demograph_MGB. 
missing_ids = notes_mgb[~notes_mgb['BDSPPatientID'].isin(df_demograph_MGB['BDSPPatientID'])]
if not missing_ids.empty:
    print("Warning: Some BDSPPatientID values in notes_mgb do not have corresponding entries in df_demograph_MGB:")
    print(missing_ids[['BDSPPatientID']])

#Filter the df_demograph_MGB to include only those IDs present in notes_mgb.
filtered_demograph = df_demograph_MGB[df_demograph_MGB['BDSPPatientID'].isin(notes_mgb['BDSPPatientID'])]

#Merge the filtered demographic DataFrame with notes_mgb. 
merged_df = pd.merge(notes_mgb, filtered_demograph, on='BDSPPatientID', how='left')

#Check for BDSPPatientID with missing DateOfBirth in merged_df.
missing_date_of_birth = merged_df[merged_df['DateOfBirth'].isna()]
if not missing_date_of_birth.empty:
    print("Warning: Some BDSPPatientID values in merged_df are missing DateOfBirth:")
    print(missing_date_of_birth[['BDSPPatientID']])

#Convert ContactDate and DateOfBirth to datetime objects. 
merged_df['ContactDate'] = pd.to_datetime(merged_df['ContactDate'], errors='coerce')
merged_df['DateOfBirth'] = pd.to_datetime(merged_df['DateOfBirth'], errors='coerce')

#Calculate Age as the difference between ContactDate and DateOfBirth.
merged_df['Age'] = (merged_df['ContactDate'] - merged_df['DateOfBirth']).dt.days // 365

#Check for BDSPPatientID with missing Age after the calculation.
missing_age = merged_df[merged_df['Age'].isna()]
if not missing_age.empty:
    print("Warning: Some BDSPPatientID values in merged_df have missing Age after calculation:")
    print(missing_age[['BDSPPatientID']])

#Save the final merged DataFrame with age calculation to a new CSV file.
final_file_path = "/home/gregory178/Desktop/NAX project/NAX_SDH/notes_mgb_with_demographics_and_age.csv"
merged_df.to_csv(final_file_path, index=False)
print(f"Final merged DataFrame with demographics and age calculation saved successfully to {final_file_path}.")


Final merged DataFrame with demographics and age calculation saved successfully to /home/gregory178/Desktop/NAX project/NAX_SDH/notes_mgb_with_demographics_and_age.csv.


In [21]:
#Creating the positve and negative icd lists. This step is essential for cohort reconstruction later. 

#Read the CSV files into DataFrames.
bidmc_df = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/notes_bidmc_with_demographics_and_birth_dates_and_age.csv")
bidmc_df_pos=pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/patientIDs_ICD_plus_SDH_BI.csv")
mgb_df = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/notes_mgb_with_demographics_and_age.csv")
mgb_df_pos = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/patientIDs_ICD_plus_SDH_MGB.csv")

#Find the matching and non-matching BDSPPatientID values.
matching_ids_BI = set(bidmc_df['BDSPPatientID']).intersection(set(bidmc_df_pos['BDSPPatientID']))
non_matching_ids_BI = set(bidmc_df['BDSPPatientID']).difference(set(bidmc_df_pos['BDSPPatientID']))

#Create the DataFrames for the matching and non-matching entries. 
bidmc_df_matching = bidmc_df[bidmc_df['BDSPPatientID'].isin(matching_ids_BI)]
bidmc_df_non_matching = bidmc_df[bidmc_df['BDSPPatientID'].isin(non_matching_ids_BI)]

#Save these DataFrames to new CSV files. 
bidmc_df_matching.to_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/bidmc_pos_icd.csv", index=False)
bidmc_df_non_matching.to_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/bidmc_neg_icd.csv", index=False)

#Find the matching and non-matching BDSPPatientID values. 
matching_ids = set(mgb_df['BDSPPatientID']).intersection(set(mgb_df_pos['BDSPPatientID']))
non_matching_ids = set(mgb_df['BDSPPatientID']).difference(set(mgb_df_pos['BDSPPatientID']))

#Create DataFrames for the matching and non-matching entries. 
mgb_df_matching = mgb_df[mgb_df['BDSPPatientID'].isin(matching_ids)]
mgb_df_non_matching = mgb_df[mgb_df['BDSPPatientID'].isin(non_matching_ids)]

#Save these DataFrames to new CSV files. 
mgb_df_matching.to_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/mgb_pos_icd.csv", index=False)
mgb_df_non_matching.to_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/mgb_neg_icd.csv", index=False)



In [22]:
#Filter out the demographics for each of the patients in the feature matrix. 

#Merge MGB_pos. 
mgb_pos=pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/mgb_pos_icd.csv")
matrix=pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/Complete_merged_feature_matrix_notes_CPT_and_ICD_.csv")

#Merge the two DataFrames on the BDSPPatientID column with an inner join
merged_df = pd.merge(mgb_pos, matrix[['BDSPPatientID', 'annot']], on='BDSPPatientID', how='inner')

#Display the result to verify
print(merged_df.head())
merged_df.to_csv()
merged_df.to_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/mgb_pos_icd.csv", index=False)


#Merge MGB minus. 
mgb_minus=pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/mgb_neg_icd.csv")
matrix=pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/Complete_merged_feature_matrix_notes_CPT_and_ICD_.csv")

#Merge the two DataFrames on the BDSPPatientID column with an inner join
merged_df = pd.merge(mgb_minus, matrix[['BDSPPatientID', 'annot']], on='BDSPPatientID', how='inner')

#Display the result to verify
print(merged_df.head())
merged_df.to_csv()
merged_df.to_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/mgb_neg_icd.csv", index=False)


#Merge BIDMC_plus. 
BIDMC_plus=pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/bidmc_pos_icd.csv")
matrix=pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/Complete_merged_feature_matrix_notes_CPT_and_ICD_.csv")

#Merge the two DataFrames on the BDSPPatientID column with an inner join
merged_df = pd.merge(BIDMC_plus, matrix[['BDSPPatientID', 'annot']], on='BDSPPatientID', how='inner')

#Display the result to verify
print(merged_df.head())
merged_df.to_csv()
merged_df.to_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/bidmc_pos_icd.csv", index=False)


#Merge BIDMC_minus. 
BIDMC_minus=pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/bidmc_neg_icd.csv")
matrix=pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/Complete_merged_feature_matrix_notes_CPT_and_ICD_.csv")

#Merge the two DataFrames on the BDSPPatientID column with an inner join.
merged_df = pd.merge(BIDMC_minus, matrix[['BDSPPatientID', 'annot']], on='BDSPPatientID', how='inner')

#Display the result to verify.
print(merged_df.head())
merged_df.to_csv()
merged_df.to_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/bidmc_neg_icd.csv", index=False)

   BDSPPatientID ContactDate Site                PatientRace EthnicGroupDSC  \
0      120109726  2019-12-01  MGB                      White   Not Hispanic   
1      111971091  2021-05-30  MGB  Black or African American   Not Hispanic   
2      114651683  2022-03-09  MGB                      White   Not Hispanic   
3      115288640  2021-03-02  MGB                      White   Not Hispanic   
4      115389340  2019-12-02  MGB                      White   Not Hispanic   

   SexDSC DateOfBirth  Age  annot  
0  Female  1935-10-15   84      1  
1  Female  1946-08-12   74      1  
2    Male  1953-02-22   69      1  
3    Male  1950-12-08   70      1  
4    Male  1941-03-13   78      1  
   BDSPPatientID ContactDate Site PatientRace EthnicGroupDSC  SexDSC  \
0      116291874  2017-06-12  MGB       Asian   Not Hispanic  Female   
1      114884810  2016-07-21  MGB       White   Not Hispanic    Male   
2      114130469  2016-06-26  MGB       White   Not Hispanic  Female   
3      113772722  202

In [23]:
#Calculate the demographics for the ICD+ group in MGB. 

#Load the new CSV file for further analysis
MGB_Plus = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/mgb_pos_icd.csv")

#Analyze the 'annot' column
total_rows = len(MGB_Plus)
annot_ones = MGB_Plus['annot'].eq(1).sum()  
percent_annot_ones = (annot_ones / total_rows) * 100 if total_rows > 0 else 0

#Print the count and percentage of rows with 'annot' equal to 1.
print(f"Count of rows where 'annot' is 1: {annot_ones}")
print(f"Percentage of rows where 'annot' is 1: {percent_annot_ones:.2f}%")

#Extract the 'age' column.
age = MGB_Plus['Age'].dropna()  

#Calculate the mean age. 
mean_age = age.mean()

#Calculate the standard deviation. 
std_dev_age = age.std()

#Calculate the min and max.
min_age = age.min()
max_age = age.max()

#Calculate IQR (Interquartile Range). 
Q1 = age.quantile(0.25)
Q3 = age.quantile(0.75)
IQR = Q3 - Q1

#Print the results.
print(f"Mean Age Both Hospitals: {mean_age}")
print(f"Standard Deviation of Age Both Hospitals: {std_dev_age}")
print(f"Minimum Age Both Hospitals: {min_age}")
print(f"Maximum Age Both Hospitals: {max_age}")
print(f"Interquartile Range (IQR) of Age Both Hospitals: {IQR}")
print(f"Quartile 1 of age Both Hospitals: {Q1}")
print(f"Quartile 3 of age Both Hospitals: {Q3}")

#Analyze 'SexDSC' column.
sex_counts = MGB_Plus['SexDSC'].value_counts()  
sex_percentages = MGB_Plus['SexDSC'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Male and Female:")
for sex in sex_counts.index:
    count = sex_counts[sex]
    percent = sex_percentages[sex]
    print(f"{sex}: {count} ({percent:.2f}%)")

#Define the updated mapping for 'PatientRace'.
def map_race(race):
    if pd.isna(race):  
        return "UNKNOWN/NOT SPECIFIED"
    race = race.strip()  
    if race in ["WHITE","White"]:
        return "WHITE"
    elif race in ["BLACK/AFRICAN AMERICAN","Black or African American"]:
        return "BLACK/AFRICAN AMERICAN"
    elif race in ["ASIAN","Asian"]:
        return "ASIAN"
    elif race in ["NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER", "Native Hawaiian or Other Pacific Islander"]:
        return "NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER"
    elif race in ["AMERICAN INDIAN/ALASKA NATIVE", "American Indian or Alaska Native"]:
        return "AMERICAN INDIAN/ALASKA NATIVE"
    elif race in ["DECLINED TO ANSWER","Declined"]:
        return "UNKNOWN/NOT SPECIFIED"
    else:
        return "OTHER"

#Apply the mapping to the 'PatientRace' column.
MGB_Plus['PatientRace'] = MGB_Plus['PatientRace'].map(map_race)

#Recalculate and print counts and percentages for the modified 'PatientRace' column.
race_counts = MGB_Plus['PatientRace'].value_counts()  
race_percentages = MGB_Plus['PatientRace'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Each Race:")
for race in race_counts.index:
    count = race_counts[race]
    percent = race_percentages[race]
    print(f"{race}: {count} ({percent:.2f}%)")

#Define the mapping for 'EthnicGroupDSC'.
def map_ethnicity(ethnicity):
    if pd.isna(ethnicity):  
        return "Unknown"
    ethnicity = ethnicity.strip()  
    if ethnicity in ["OTHER ETHNICITY", "OTHER", "PREFER NOT TO SAY", "UNKNOWN/NOT SPECIFIED","UNABLE TO OBTAIN","DECLINED TO ANSWER", 
                     "LATIN AMERICAN", "SOUTH AMERICAN (NOT OTHERWISE SPECIFIED)","Unavailable", "Prefer not to say/Decline"]:
        return "Unknown"
    elif ethnicity in ["SALVADORIAN", "DOMINICAN", "PUERTO RICAN", "GUATEMALAN", "MEXICAN", "COLUMBIAN", "SPANISH", "ARGENTINIAN",
                       "ARGENTINE", "PERUVIAN", "VENEZUELAN", "CHILEAN", "ECUADORIAN", "COSTA RICAN", "CUBAN", "BOLIVIAN", "HONDURAN",
                       "PARAGUAYAN", "NICARAGUAN", "PANAMANIAN", "URUGUAYAN", "EQUATORIAL GUINEAN","MEXICAN, MEXICAN AMERICAN, CHICANO",
                        "SALVADORAN", "COLOMBIAN", "CARIBBEAN ISLAND", "Hispanic" ]:
        return "Hispanic"
    else:
        return "Not Hispanic"

#Apply the mapping to the 'EthnicGroupDSC' column. 
MGB_Plus['EthnicGroupDSC'] = MGB_Plus['EthnicGroupDSC'].map(map_ethnicity)

#Recalculate and print counts and percentages for the modified 'EthnicGroupDSC' column
ethnicity_counts = MGB_Plus['EthnicGroupDSC'].value_counts()  
ethnicity_percentages = MGB_Plus['EthnicGroupDSC'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Each Ethnicity:")
for ethnicity in ethnicity_counts.index:
    count = ethnicity_counts[ethnicity]
    percent = ethnicity_percentages[ethnicity]
    print(f"{ethnicity}: {count} ({percent:.2f}%)")

#Identify and print outliers. 
outliers = MGB_Plus[(MGB_Plus['Age'] > 120)]
print("\nBDSPPatientIDs, Ages, ContactDates, and DateOfBirths for patients with age > 120:")
for _, row in outliers.iterrows():
    print(f"BDSPPatientID: {row['BDSPPatientID']}, Age: {row['Age']}, ContactDate: {row['ContactDate']}, DateOfBirth: {row['DateOfBirth']}")


Count of rows where 'annot' is 1: 370
Percentage of rows where 'annot' is 1: 49.33%
Mean Age Both Hospitals: 68.22
Standard Deviation of Age Both Hospitals: 17.17912773997975
Minimum Age Both Hospitals: 0
Maximum Age Both Hospitals: 102
Interquartile Range (IQR) of Age Both Hospitals: 21.0
Quartile 1 of age Both Hospitals: 60.0
Quartile 3 of age Both Hospitals: 81.0

Count and Percentage of Male and Female:
Male: 497 (66.27%)
Female: 253 (33.73%)

Count and Percentage of Each Race:
WHITE: 587 (78.27%)
OTHER: 64 (8.53%)
BLACK/AFRICAN AMERICAN: 60 (8.00%)
ASIAN: 26 (3.47%)
UNKNOWN/NOT SPECIFIED: 12 (1.60%)
AMERICAN INDIAN/ALASKA NATIVE: 1 (0.13%)

Count and Percentage of Each Ethnicity:
Not Hispanic: 674 (89.87%)
Hispanic: 47 (6.27%)
Unknown: 29 (3.87%)

BDSPPatientIDs, Ages, ContactDates, and DateOfBirths for patients with age > 120:


In [24]:
#Print out demographic information for the ICD- group in MGB. 
MGB_Minus = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/mgb_neg_icd.csv")

#Analyze the 'annot' column
total_rows = len(MGB_Minus)
annot_ones = MGB_Minus['annot'].eq(1).sum()  
percent_annot_ones = (annot_ones / total_rows) * 100 if total_rows > 0 else 0

#Print the count and percentage of rows with 'annot' equal to 1. 
print(f"Count of rows where 'annot' is 1: {annot_ones}")
print(f"Percentage of rows where 'annot' is 1: {percent_annot_ones:.2f}%")

#Extract the 'age' column.
age = MGB_Minus['Age'].dropna()  

#Calculate the mean age. 
mean_age = age.mean()

#Calculate the standard deviation. 
std_dev_age = age.std()

#Calculate the min and max.
min_age = age.min()
max_age = age.max()

#Calculate the IQR (Interquartile Range). 
Q1 = age.quantile(0.25)
Q3 = age.quantile(0.75)
IQR = Q3 - Q1

#Print the results. 
print(f"Mean Age Both Hospitals: {mean_age}")
print(f"Standard Deviation of Age Both Hospitals: {std_dev_age}")
print(f"Minimum Age Both Hospitals: {min_age}")
print(f"Maximum Age Both Hospitals: {max_age}")
print(f"Interquartile Range (IQR) of Age Both Hospitals: {IQR}")
print(f"Quartile 1 of age Both Hospitals: {Q1}")
print(f"Quartile 3 of age Both Hospitals: {Q3}")


#Analyze the 'SexDSC' column. 
sex_counts = MGB_Minus['SexDSC'].value_counts()  
sex_percentages = MGB_Minus['SexDSC'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Male and Female:")
for sex in sex_counts.index:
    count = sex_counts[sex]
    percent = sex_percentages[sex]
    print(f"{sex}: {count} ({percent:.2f}%)")

#Define the updated mapping for 'PatientRace'. 
def map_race(race):
    if pd.isna(race):  
        return "UNKNOWN/NOT SPECIFIED"
    race = race.strip()  
    if race in ["WHITE","White"]:
        return "WHITE"
    elif race in ["BLACK/AFRICAN AMERICAN","Black or African American"]:
        return "BLACK/AFRICAN AMERICAN"
    elif race in ["ASIAN","Asian"]:
        return "ASIAN"
    elif race in ["NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER", "Native Hawaiian or Other Pacific Islander"]:
        return "NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER"
    elif race in ["AMERICAN INDIAN/ALASKA NATIVE", "American Indian or Alaska Native"]:
        return "AMERICAN INDIAN/ALASKA NATIVE"
    elif race in ["DECLINED TO ANSWER","Declined"]:
        return "UNKNOWN/NOT SPECIFIED"
    else:
        return "OTHER"


#Apply the mapping to the 'PatientRace' column.
MGB_Minus['PatientRace'] = MGB_Minus['PatientRace'].map(map_race)

#Recalculate and print counts and percentages for the modified 'PatientRace' column.
race_counts = MGB_Minus['PatientRace'].value_counts()  
race_percentages = MGB_Minus['PatientRace'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Each Race:")
for race in race_counts.index:
    count = race_counts[race]
    percent = race_percentages[race]
    print(f"{race}: {count} ({percent:.2f}%)")

#Define the mapping for 'EthnicGroupDSC'.
def map_ethnicity(ethnicity):
    if pd.isna(ethnicity):  
        return "Unknown"
    ethnicity = ethnicity.strip()  
    if ethnicity in ["OTHER ETHNICITY", "OTHER", "PREFER NOT TO SAY", "UNKNOWN/NOT SPECIFIED","UNABLE TO OBTAIN","DECLINED TO ANSWER", 
                     "LATIN AMERICAN", "SOUTH AMERICAN (NOT OTHERWISE SPECIFIED)","Unavailable", "Prefer not to say/Decline"]:
        return "Unknown"
    elif ethnicity in ["SALVADORIAN", "DOMINICAN", "PUERTO RICAN", "GUATEMALAN", "MEXICAN", "COLUMBIAN", "SPANISH", "ARGENTINIAN",
                       "ARGENTINE", "PERUVIAN", "VENEZUELAN", "CHILEAN", "ECUADORIAN", "COSTA RICAN", "CUBAN", "BOLIVIAN", "HONDURAN",
                       "PARAGUAYAN", "NICARAGUAN", "PANAMANIAN", "URUGUAYAN", "EQUATORIAL GUINEAN","MEXICAN, MEXICAN AMERICAN, CHICANO",
                        "SALVADORAN", "COLOMBIAN", "CARIBBEAN ISLAND", "Hispanic" ]:
        return "Hispanic"
    else:
        return "Not Hispanic"
    
#Apply the mapping to the 'EthnicGroupDSC' column.
MGB_Minus['EthnicGroupDSC'] = MGB_Minus['EthnicGroupDSC'].map(map_ethnicity)

#Recalculate and print counts and percentages for the modified 'EthnicGroupDSC' column
ethnicity_counts = MGB_Minus['EthnicGroupDSC'].value_counts()  
ethnicity_percentages = MGB_Minus['EthnicGroupDSC'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Each Ethnicity:")
for ethnicity in ethnicity_counts.index:
    count = ethnicity_counts[ethnicity]
    percent = ethnicity_percentages[ethnicity]
    print(f"{ethnicity}: {count} ({percent:.2f}%)")

outliers = MGB_Minus[(MGB_Minus['Age'] > 120)]
print("\nBDSPPatientIDs, Ages, ContactDates, and DateOfBirths for patients with age > 100 or < 5:")
for _, row in outliers.iterrows():
    print(f"BDSPPatientID: {row['BDSPPatientID']}, Age: {row['Age']}, ContactDate: {row['ContactDate']}, DateOfBirth: {row['DateOfBirth']}")


Count of rows where 'annot' is 1: 3
Percentage of rows where 'annot' is 1: 0.40%
Mean Age Both Hospitals: 55.15754339118825
Standard Deviation of Age Both Hospitals: 22.585470753211474
Minimum Age Both Hospitals: 0
Maximum Age Both Hospitals: 96
Interquartile Range (IQR) of Age Both Hospitals: 33.0
Quartile 1 of age Both Hospitals: 39.0
Quartile 3 of age Both Hospitals: 72.0

Count and Percentage of Male and Female:
Female: 379 (50.60%)
Male: 370 (49.40%)

Count and Percentage of Each Race:
WHITE: 569 (75.97%)
OTHER: 87 (11.62%)
BLACK/AFRICAN AMERICAN: 61 (8.14%)
ASIAN: 19 (2.54%)
UNKNOWN/NOT SPECIFIED: 11 (1.47%)
AMERICAN INDIAN/ALASKA NATIVE: 1 (0.13%)
NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER: 1 (0.13%)

Count and Percentage of Each Ethnicity:
Not Hispanic: 624 (83.31%)
Hispanic: 71 (9.48%)
Unknown: 54 (7.21%)

BDSPPatientIDs, Ages, ContactDates, and DateOfBirths for patients with age > 100 or < 5:


In [25]:
#Calculate the demographic information for ICD+ for BIDMC. 
BIDMC_Plus = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/bidmc_pos_icd.csv")

#Analyze the 'annot' column. 
total_rows = len(BIDMC_Plus)
annot_ones = BIDMC_Plus['annot'].eq(1).sum()  
percent_annot_ones = (annot_ones / total_rows) * 100 if total_rows > 0 else 0

#Print count and percentage of rows with 'annot' equal to 1. 
print(f"Count of rows where 'annot' is 1: {annot_ones}")
print(f"Percentage of rows where 'annot' is 1: {percent_annot_ones:.2f}%")

#Extract the 'age' column. 
age = BIDMC_Plus['Age'].dropna()  

#Calculate mean age. 
mean_age = age.mean()

#Calculate standard deviation. 
std_dev_age = age.std()

#Calculate min and max.
min_age = age.min()
max_age = age.max()

#Calculate IQR (Interquartile Range).
Q1 = age.quantile(0.25)
Q3 = age.quantile(0.75)
IQR = Q3 - Q1

#Print the results.
print(f"Mean Age Both Hospitals: {mean_age}")
print(f"Standard Deviation of Age Both Hospitals: {std_dev_age}")
print(f"Minimum Age Both Hospitals: {min_age}")
print(f"Maximum Age Both Hospitals: {max_age}")
print(f"Interquartile Range (IQR) of Age Both Hospitals: {IQR}")
print(f"Quartile 1 of age Both Hospitals: {Q1}")
print(f"Quartile 3 of age Both Hospitals: {Q3}")


#Analyze 'SexDSC' column.
sex_counts = BIDMC_Plus['SexDSC'].value_counts()  
sex_percentages = BIDMC_Plus['SexDSC'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Male and Female:")
for sex in sex_counts.index:
    count = sex_counts[sex]
    percent = sex_percentages[sex]
    print(f"{sex}: {count} ({percent:.2f}%)")

#Define the updated mapping for 'PatientRace'.
def map_race(race):
    if pd.isna(race):  
        return "UNKNOWN/NOT SPECIFIED"
    race = race.strip()  
    if race in ["WHITE","White"]:
        return "WHITE"
    elif race in ["BLACK/AFRICAN AMERICAN","Black or African American"]:
        return "BLACK/AFRICAN AMERICAN"
    elif race in ["ASIAN","Asian"]:
        return "ASIAN"
    elif race in ["NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER", "Native Hawaiian or Other Pacific Islander"]:
        return "NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER"
    elif race in ["AMERICAN INDIAN/ALASKA NATIVE", "American Indian or Alaska Native"]:
        return "AMERICAN INDIAN/ALASKA NATIVE"
    elif race in ["DECLINED TO ANSWER","Declined"]:
        return "UNKNOWN/NOT SPECIFIED"
    else:
        return "OTHER"


#Apply the mapping to the 'PatientRace' column.
BIDMC_Plus['PatientRace'] = BIDMC_Plus['PatientRace'].map(map_race)

#Recalculate and print counts and percentages for the modified 'PatientRace' column.
race_counts = BIDMC_Plus['PatientRace'].value_counts()  
race_percentages = BIDMC_Plus['PatientRace'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Each Race:")
for race in race_counts.index:
    count = race_counts[race]
    percent = race_percentages[race]
    print(f"{race}: {count} ({percent:.2f}%)")

#Define the mapping for 'EthnicGroupDSC'.
def map_ethnicity(ethnicity):
    if pd.isna(ethnicity): 
        return "Unknown"
    ethnicity = ethnicity.strip()  
    if ethnicity in ["OTHER ETHNICITY", "OTHER", "PREFER NOT TO SAY", "UNKNOWN/NOT SPECIFIED","UNABLE TO OBTAIN","DECLINED TO ANSWER", 
                     "LATIN AMERICAN", "SOUTH AMERICAN (NOT OTHERWISE SPECIFIED)","Unavailable", "Prefer not to say/Decline"]:
        return "Unknown"
    elif ethnicity in ["SALVADORIAN", "DOMINICAN", "PUERTO RICAN", "GUATEMALAN", "MEXICAN", "COLUMBIAN", "SPANISH", "ARGENTINIAN",
                       "ARGENTINE", "PERUVIAN", "VENEZUELAN", "CHILEAN", "ECUADORIAN", "COSTA RICAN", "CUBAN", "BOLIVIAN", "HONDURAN",
                       "PARAGUAYAN", "NICARAGUAN", "PANAMANIAN", "URUGUAYAN", "EQUATORIAL GUINEAN","MEXICAN, MEXICAN AMERICAN, CHICANO",
                        "SALVADORAN", "COLOMBIAN", "CARIBBEAN ISLAND", "Hispanic" ]:
        return "Hispanic"
    else:
        return "Not Hispanic"
    
#Apply the mapping to the 'EthnicGroupDSC' column.
BIDMC_Plus['EthnicGroupDSC'] = BIDMC_Plus['EthnicGroupDSC'].map(map_ethnicity)

#Recalculate and print counts and percentages for the modified 'EthnicGroupDSC' column.
ethnicity_counts = BIDMC_Plus['EthnicGroupDSC'].value_counts()  
ethnicity_percentages = BIDMC_Plus['EthnicGroupDSC'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Each Ethnicity:")
for ethnicity in ethnicity_counts.index:
    count = ethnicity_counts[ethnicity]
    percent = ethnicity_percentages[ethnicity]
    print(f"{ethnicity}: {count} ({percent:.2f}%)")

outliers = BIDMC_Plus[(BIDMC_Plus['Age'] > 120)]
print("\nBDSPPatientIDs, Ages, ContactDates, and DateOfBirths for patients with age > 100 or < 5:")
for _, row in outliers.iterrows():
    print(f"BDSPPatientID: {row['BDSPPatientID']}, Age: {row['Age']}, ContactDate: {row['ContactDate']}, DateOfBirth: {row['DateOfBirth']}")


Count of rows where 'annot' is 1: 341
Percentage of rows where 'annot' is 1: 45.47%
Mean Age Both Hospitals: 70.976
Standard Deviation of Age Both Hospitals: 16.51400538427236
Minimum Age Both Hospitals: 21
Maximum Age Both Hospitals: 100
Interquartile Range (IQR) of Age Both Hospitals: 21.0
Quartile 1 of age Both Hospitals: 62.0
Quartile 3 of age Both Hospitals: 83.0

Count and Percentage of Male and Female:
Male: 442 (58.93%)
Female: 308 (41.07%)

Count and Percentage of Each Race:
WHITE: 552 (73.60%)
OTHER: 70 (9.33%)
BLACK/AFRICAN AMERICAN: 66 (8.80%)
UNKNOWN/NOT SPECIFIED: 33 (4.40%)
ASIAN: 27 (3.60%)
NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER: 1 (0.13%)
AMERICAN INDIAN/ALASKA NATIVE: 1 (0.13%)

Count and Percentage of Each Ethnicity:
Not Hispanic: 583 (77.73%)
Unknown: 135 (18.00%)
Hispanic: 32 (4.27%)

BDSPPatientIDs, Ages, ContactDates, and DateOfBirths for patients with age > 100 or < 5:


In [26]:
#Calculate the demographics information for the ICD- group for BIDMC. 
BIDMC_Minus = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/bidmc_neg_icd.csv")

#Analyze the 'annot' column. 
total_rows = len(BIDMC_Minus)
annot_ones = BIDMC_Minus['annot'].eq(1).sum()  
percent_annot_ones = (annot_ones / total_rows) * 100 if total_rows > 0 else 0

#Print the count and percentage of rows with 'annot' equal to 1. 
print(f"Count of rows where 'annot' is 1: {annot_ones}")
print(f"Percentage of rows where 'annot' is 1: {percent_annot_ones:.2f}%")

#Extract the 'age' column. 
age = BIDMC_Minus['Age'].dropna()  

#Calculate the mean age. 
mean_age = age.mean()

#Calculate the standard deviation. 
std_dev_age = age.std()

#Calculate the min and max. 
min_age = age.min()
max_age = age.max()

#Calculate IQR (Interquartile Range). 
Q1 = age.quantile(0.25)
Q3 = age.quantile(0.75)
IQR = Q3 - Q1

#Print the results. 
print(f"Mean Age Both Hospitals: {mean_age}")
print(f"Standard Deviation of Age Both Hospitals: {std_dev_age}")
print(f"Minimum Age Both Hospitals: {min_age}")
print(f"Maximum Age Both Hospitals: {max_age}")
print(f"Interquartile Range (IQR) of Age Both Hospitals: {IQR}")
print(f"Quartile 1 of age Both Hospitals: {Q1}")
print(f"Quartile 3 of age Both Hospitals: {Q3}")


#Analyze 'SexDSC' column. 
sex_counts = BIDMC_Minus['SexDSC'].value_counts()  
sex_percentages = BIDMC_Minus['SexDSC'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Male and Female:")
for sex in sex_counts.index:
    count = sex_counts[sex]
    percent = sex_percentages[sex]
    print(f"{sex}: {count} ({percent:.2f}%)")

#Define the updated mapping for 'PatientRace'
def map_race(race):
    if pd.isna(race):  
        return "UNKNOWN/NOT SPECIFIED"
    race = race.strip()  
    if race in ["WHITE","White"]:
        return "WHITE"
    elif race in ["BLACK/AFRICAN AMERICAN","Black or African American"]:
        return "BLACK/AFRICAN AMERICAN"
    elif race in ["ASIAN","Asian"]:
        return "ASIAN"
    elif race in ["NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER", "Native Hawaiian or Other Pacific Islander"]:
        return "NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER"
    elif race in ["AMERICAN INDIAN/ALASKA NATIVE", "American Indian or Alaska Native"]:
        return "AMERICAN INDIAN/ALASKA NATIVE"
    elif race in ["DECLINED TO ANSWER","Declined"]:
        return "UNKNOWN/NOT SPECIFIED"
    else:
        return "OTHER"


#Apply the mapping to the 'PatientRace' column.
BIDMC_Minus['PatientRace'] = BIDMC_Minus['PatientRace'].map(map_race)

#Recalculate and print counts and percentages for the modified 'PatientRace' column.
race_counts = BIDMC_Minus['PatientRace'].value_counts()  
race_percentages = BIDMC_Minus['PatientRace'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Each Race:")
for race in race_counts.index:
    count = race_counts[race]
    percent = race_percentages[race]
    print(f"{race}: {count} ({percent:.2f}%)")

#Define the mapping for 'EthnicGroupDSC'.
def map_ethnicity(ethnicity):
    if pd.isna(ethnicity):  
        return "Unknown"
    ethnicity = ethnicity.strip()  
    if ethnicity in ["OTHER ETHNICITY", "OTHER", "PREFER NOT TO SAY", "UNKNOWN/NOT SPECIFIED","UNABLE TO OBTAIN","DECLINED TO ANSWER", 
                     "LATIN AMERICAN", "SOUTH AMERICAN (NOT OTHERWISE SPECIFIED)","Unavailable", "Prefer not to say/Decline"]:
        return "Unknown"
    elif ethnicity in ["SALVADORIAN", "DOMINICAN", "PUERTO RICAN", "GUATEMALAN", "MEXICAN", "COLUMBIAN", "SPANISH", "ARGENTINIAN",
                       "ARGENTINE", "PERUVIAN", "VENEZUELAN", "CHILEAN", "ECUADORIAN", "COSTA RICAN", "CUBAN", "BOLIVIAN", "HONDURAN",
                       "PARAGUAYAN", "NICARAGUAN", "PANAMANIAN", "URUGUAYAN", "EQUATORIAL GUINEAN","MEXICAN, MEXICAN AMERICAN, CHICANO",
                        "SALVADORAN", "COLOMBIAN", "CARIBBEAN ISLAND", "Hispanic" ]:
        return "Hispanic"
    else:
        return "Not Hispanic"
    
#Apply the mapping to the 'EthnicGroupDSC' column.
BIDMC_Minus['EthnicGroupDSC'] = BIDMC_Minus['EthnicGroupDSC'].map(map_ethnicity)

#Recalculate and print counts and percentages for the modified 'EthnicGroupDSC' column
ethnicity_counts = BIDMC_Minus['EthnicGroupDSC'].value_counts()  
ethnicity_percentages = BIDMC_Minus['EthnicGroupDSC'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Each Ethnicity:")
for ethnicity in ethnicity_counts.index:
    count = ethnicity_counts[ethnicity]
    percent = ethnicity_percentages[ethnicity]
    print(f"{ethnicity}: {count} ({percent:.2f}%)")
outliers = BIDMC_Minus[(BIDMC_Minus['Age'] > 120)]
print("\nBDSPPatientIDs, Ages, ContactDates, and DateOfBirths for patients with age > 100 or < 5:")
for _, row in outliers.iterrows():
    print(f"BDSPPatientID: {row['BDSPPatientID']}, Age: {row['Age']}, ContactDate: {row['ContactDate']}, DateOfBirth: {row['DateOfBirth']}")


Count of rows where 'annot' is 1: 0
Percentage of rows where 'annot' is 1: 0.00%
Mean Age Both Hospitals: 59.01866666666667
Standard Deviation of Age Both Hospitals: 17.625754106779826
Minimum Age Both Hospitals: 5
Maximum Age Both Hospitals: 101
Interquartile Range (IQR) of Age Both Hospitals: 24.0
Quartile 1 of age Both Hospitals: 48.0
Quartile 3 of age Both Hospitals: 72.0

Count and Percentage of Male and Female:
Female: 440 (58.67%)
Male: 310 (41.33%)

Count and Percentage of Each Race:
WHITE: 481 (64.13%)
BLACK/AFRICAN AMERICAN: 110 (14.67%)
OTHER: 73 (9.73%)
UNKNOWN/NOT SPECIFIED: 48 (6.40%)
ASIAN: 36 (4.80%)
AMERICAN INDIAN/ALASKA NATIVE: 2 (0.27%)

Count and Percentage of Each Ethnicity:
Not Hispanic: 519 (69.20%)
Unknown: 211 (28.13%)
Hispanic: 20 (2.67%)

BDSPPatientIDs, Ages, ContactDates, and DateOfBirths for patients with age > 100 or < 5:


In [27]:
#Concatenate the groups to create a total hospital demographics group for both hospitals. 
MGB_Plus = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/mgb_pos_icd.csv")
MGB_Minus = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/mgb_neg_icd.csv")
BIDMC_Plus = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/bidmc_pos_icd.csv")
BIDMC_Minus = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/bidmc_neg_icd.csv")

MGB_total = pd.concat([MGB_Plus, MGB_Minus])
MGB_total.to_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/MGB_total_ICD.csv", index=False)

BIDMC_total=pd.concat([BIDMC_Plus, BIDMC_Minus])
BIDMC_total.to_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/BIDMC_total_ICD.csv", index=False)



In [28]:
#Print out the demographic information for BIDMC. 
BIDMC_Total = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/BIDMC_total_ICD.csv")

#Analyze the 'annot' column. 
total_rows = len(BIDMC_Total)
annot_ones = BIDMC_Total['annot'].eq(1).sum()  
percent_annot_ones = (annot_ones / total_rows) * 100 if total_rows > 0 else 0

#Print count and percentage of rows with 'annot' equal to 1.
print(f"Count of rows where 'annot' is 1: {annot_ones}")
print(f"Percentage of rows where 'annot' is 1: {percent_annot_ones:.2f}%")

#Extract the 'age' column.
age = BIDMC_Total['Age'].dropna()  

#Calculate mean age.
mean_age = age.mean()

#Calculate standard deviation.
std_dev_age = age.std()

#Calculate min and max.
min_age = age.min()
max_age = age.max()

#Calculate IQR (Interquartile Range).
Q1 = age.quantile(0.25)
Q3 = age.quantile(0.75)
IQR = Q3 - Q1

#Print the results.
print(f"Mean Age Both Hospitals: {mean_age}")
print(f"Standard Deviation of Age Both Hospitals: {std_dev_age}")
print(f"Minimum Age Both Hospitals: {min_age}")
print(f"Maximum Age Both Hospitals: {max_age}")
print(f"Interquartile Range (IQR) of Age Both Hospitals: {IQR}")
print(f"Quartile 1 of age Both Hospitals: {Q1}")
print(f"Quartile 3 of age Both Hospitals: {Q3}")


#Analyze 'SexDSC' column.
sex_counts = BIDMC_Total['SexDSC'].value_counts()  
sex_percentages = BIDMC_Total['SexDSC'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Male and Female:")
for sex in sex_counts.index:
    count = sex_counts[sex]
    percent = sex_percentages[sex]
    print(f"{sex}: {count} ({percent:.2f}%)")

#Define the updated mapping for 'PatientRace'.
def map_race(race):
    if pd.isna(race):  
        return "UNKNOWN/NOT SPECIFIED"
    race = race.strip()  
    if race in ["WHITE","White"]:
        return "WHITE"
    elif race in ["BLACK/AFRICAN AMERICAN","Black or African American"]:
        return "BLACK/AFRICAN AMERICAN"
    elif race in ["ASIAN","Asian"]:
        return "ASIAN"
    elif race in ["NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER", "Native Hawaiian or Other Pacific Islander"]:
        return "NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER"
    elif race in ["AMERICAN INDIAN/ALASKA NATIVE", "American Indian or Alaska Native"]:
        return "AMERICAN INDIAN/ALASKA NATIVE"
    elif race in ["DECLINED TO ANSWER","Declined"]:
        return "UNKNOWN/NOT SPECIFIED"
    else:
        return "OTHER"


#Apply the mapping to the 'PatientRace' column.
BIDMC_Total['PatientRace'] = BIDMC_Total['PatientRace'].map(map_race)

#Recalculate and print counts and percentages for the modified 'PatientRace' column.
race_counts = BIDMC_Total['PatientRace'].value_counts()  
race_percentages = BIDMC_Total['PatientRace'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Each Race:")
for race in race_counts.index:
    count = race_counts[race]
    percent = race_percentages[race]
    print(f"{race}: {count} ({percent:.2f}%)")

#Define the mapping for 'EthnicGroupDSC'.
def map_ethnicity(ethnicity):
    if pd.isna(ethnicity):  
        return "Unknown"
    ethnicity = ethnicity.strip()  
    if ethnicity in ["OTHER ETHNICITY", "OTHER", "PREFER NOT TO SAY", "UNKNOWN/NOT SPECIFIED","UNABLE TO OBTAIN","DECLINED TO ANSWER", 
                     "LATIN AMERICAN", "SOUTH AMERICAN (NOT OTHERWISE SPECIFIED)","Unavailable", "Prefer not to say/Decline"]:
        return "Unknown"
    elif ethnicity in ["SALVADORIAN", "DOMINICAN", "PUERTO RICAN", "GUATEMALAN", "MEXICAN", "COLUMBIAN", "SPANISH", "ARGENTINIAN",
                       "ARGENTINE", "PERUVIAN", "VENEZUELAN", "CHILEAN", "ECUADORIAN", "COSTA RICAN", "CUBAN", "BOLIVIAN", "HONDURAN",
                       "PARAGUAYAN", "NICARAGUAN", "PANAMANIAN", "URUGUAYAN", "EQUATORIAL GUINEAN","MEXICAN, MEXICAN AMERICAN, CHICANO",
                        "SALVADORAN", "COLOMBIAN", "CARIBBEAN ISLAND", "Hispanic" ]:
        return "Hispanic"
    else:
        return "Not Hispanic"
    
#Apply the mapping to the 'EthnicGroupDSC' column.
BIDMC_Total['EthnicGroupDSC'] = BIDMC_Total['EthnicGroupDSC'].map(map_ethnicity)

#Recalculate and print counts and percentages for the modified 'EthnicGroupDSC' column.
ethnicity_counts = BIDMC_Total['EthnicGroupDSC'].value_counts()  
ethnicity_percentages = BIDMC_Total['EthnicGroupDSC'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Each Ethnicity:")
for ethnicity in ethnicity_counts.index:
    count = ethnicity_counts[ethnicity]
    percent = ethnicity_percentages[ethnicity]
    print(f"{ethnicity}: {count} ({percent:.2f}%)")
outliers = BIDMC_Total[(BIDMC_Total['Age'] > 120)]
print("\nBDSPPatientIDs, Ages, ContactDates, and DateOfBirths for patients with age > 100 or < 5:")
for _, row in outliers.iterrows():
    print(f"BDSPPatientID: {row['BDSPPatientID']}, Age: {row['Age']}, ContactDate: {row['ContactDate']}, DateOfBirth: {row['DateOfBirth']}")


Count of rows where 'annot' is 1: 341
Percentage of rows where 'annot' is 1: 22.73%
Mean Age Both Hospitals: 64.99733333333333
Standard Deviation of Age Both Hospitals: 18.090425696764296
Minimum Age Both Hospitals: 5
Maximum Age Both Hospitals: 101
Interquartile Range (IQR) of Age Both Hospitals: 24.0
Quartile 1 of age Both Hospitals: 55.0
Quartile 3 of age Both Hospitals: 79.0

Count and Percentage of Male and Female:
Male: 752 (50.13%)
Female: 748 (49.87%)

Count and Percentage of Each Race:
WHITE: 1033 (68.87%)
BLACK/AFRICAN AMERICAN: 176 (11.73%)
OTHER: 143 (9.53%)
UNKNOWN/NOT SPECIFIED: 81 (5.40%)
ASIAN: 63 (4.20%)
AMERICAN INDIAN/ALASKA NATIVE: 3 (0.20%)
NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER: 1 (0.07%)

Count and Percentage of Each Ethnicity:
Not Hispanic: 1102 (73.47%)
Unknown: 346 (23.07%)
Hispanic: 52 (3.47%)

BDSPPatientIDs, Ages, ContactDates, and DateOfBirths for patients with age > 100 or < 5:


In [29]:
#Calculate and print out demographic information for MGB.
MGB_Total = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/MGB_total_ICD.csv")

# Analyze the 'annot' column.
total_rows = len(MGB_Total)
annot_ones = MGB_Total['annot'].eq(1).sum()  
percent_annot_ones = (annot_ones / total_rows) * 100 if total_rows > 0 else 0

#Print count and percentage of rows with 'annot' equal to 1.
print(f"Count of rows where 'annot' is 1: {annot_ones}")
print(f"Percentage of rows where 'annot' is 1: {percent_annot_ones:.2f}%")

#Extract the 'age' column.
age = MGB_Total['Age'].dropna()  

#Calculate the mean age. 
mean_age = age.mean()

#Calculate the standard deviation. 
std_dev_age = age.std()

#Calculate min and max. 
min_age = age.min()
max_age = age.max()

#Calculate IQR (Interquartile Range).
Q1 = age.quantile(0.25)
Q3 = age.quantile(0.75)
IQR = Q3 - Q1

#Print the results
print(f"Mean Age Both Hospitals: {mean_age}")
print(f"Standard Deviation of Age Both Hospitals: {std_dev_age}")
print(f"Minimum Age Both Hospitals: {min_age}")
print(f"Maximum Age Both Hospitals: {max_age}")
print(f"Interquartile Range (IQR) of Age Both Hospitals: {IQR}")
print(f"Quartile 1 of age Both Hospitals: {Q1}")
print(f"Quartile 3 of age Both Hospitals: {Q3}")


#Analyze 'SexDSC' column.
sex_counts = MGB_Total['SexDSC'].value_counts()  
sex_percentages = MGB_Total['SexDSC'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Male and Female:")
for sex in sex_counts.index:
    count = sex_counts[sex]
    percent = sex_percentages[sex]
    print(f"{sex}: {count} ({percent:.2f}%)")

#Define the updated mapping for 'PatientRace'
def map_race(race):
    if pd.isna(race):  
        return "UNKNOWN/NOT SPECIFIED"
    race = race.strip()  
    if race in ["WHITE","White"]:
        return "WHITE"
    elif race in ["BLACK/AFRICAN AMERICAN","Black or African American"]:
        return "BLACK/AFRICAN AMERICAN"
    elif race in ["ASIAN","Asian"]:
        return "ASIAN"
    elif race in ["NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER", "Native Hawaiian or Other Pacific Islander"]:
        return "NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER"
    elif race in ["AMERICAN INDIAN/ALASKA NATIVE", "American Indian or Alaska Native"]:
        return "AMERICAN INDIAN/ALASKA NATIVE"
    elif race in ["DECLINED TO ANSWER","Declined"]:
        return "UNKNOWN/NOT SPECIFIED"
    else:
        return "OTHER"


#Apply the mapping to the 'PatientRace' column.
MGB_Total['PatientRace'] = MGB_Total['PatientRace'].map(map_race)

#Recalculate and print counts and percentages for the modified 'PatientRace' column.
race_counts = MGB_Total['PatientRace'].value_counts()  
race_percentages = MGB_Total['PatientRace'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Each Race:")
for race in race_counts.index:
    count = race_counts[race]
    percent = race_percentages[race]
    print(f"{race}: {count} ({percent:.2f}%)")

#Define the mapping for 'EthnicGroupDSC'
def map_ethnicity(ethnicity):
    if pd.isna(ethnicity):  
        return "Unknown"
    ethnicity = ethnicity.strip()  
    if ethnicity in ["OTHER ETHNICITY", "OTHER", "PREFER NOT TO SAY", "UNKNOWN/NOT SPECIFIED","UNABLE TO OBTAIN","DECLINED TO ANSWER", 
                     "LATIN AMERICAN", "SOUTH AMERICAN (NOT OTHERWISE SPECIFIED)","Unavailable", "Prefer not to say/Decline"]:
        return "Unknown"
    elif ethnicity in ["SALVADORIAN", "DOMINICAN", "PUERTO RICAN", "GUATEMALAN", "MEXICAN", "COLUMBIAN", "SPANISH", "ARGENTINIAN",
                       "ARGENTINE", "PERUVIAN", "VENEZUELAN", "CHILEAN", "ECUADORIAN", "COSTA RICAN", "CUBAN", "BOLIVIAN", "HONDURAN",
                       "PARAGUAYAN", "NICARAGUAN", "PANAMANIAN", "URUGUAYAN", "EQUATORIAL GUINEAN","MEXICAN, MEXICAN AMERICAN, CHICANO",
                        "SALVADORAN", "COLOMBIAN", "CARIBBEAN ISLAND", "Hispanic" ]:
        return "Hispanic"
    else:
        return "Not Hispanic"
#Apply the mapping to the 'EthnicGroupDSC' column.
MGB_Total['EthnicGroupDSC'] = MGB_Total['EthnicGroupDSC'].map(map_ethnicity)

#Recalculate and print counts and percentages for the modified 'EthnicGroupDSC' column.
ethnicity_counts = MGB_Total['EthnicGroupDSC'].value_counts()  
ethnicity_percentages = MGB_Total['EthnicGroupDSC'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Each Ethnicity:")
for ethnicity in ethnicity_counts.index:
    count = ethnicity_counts[ethnicity]
    percent = ethnicity_percentages[ethnicity]
    print(f"{ethnicity}: {count} ({percent:.2f}%)")

outliers = MGB_Total[(MGB_Total['Age'] > 120)]
print("\nBDSPPatientIDs, Ages, ContactDates, and DateOfBirths for patients with age > 100 or < 5:")
for _, row in outliers.iterrows():
    print(f"BDSPPatientID: {row['BDSPPatientID']}, Age: {row['Age']}, ContactDate: {row['ContactDate']}, DateOfBirth: {row['DateOfBirth']}")


Count of rows where 'annot' is 1: 373
Percentage of rows where 'annot' is 1: 24.88%
Mean Age Both Hospitals: 61.693128752501664
Standard Deviation of Age Both Hospitals: 21.09402371762122
Minimum Age Both Hospitals: 0
Maximum Age Both Hospitals: 102
Interquartile Range (IQR) of Age Both Hospitals: 25.0
Quartile 1 of age Both Hospitals: 52.0
Quartile 3 of age Both Hospitals: 77.0

Count and Percentage of Male and Female:
Male: 867 (57.84%)
Female: 632 (42.16%)

Count and Percentage of Each Race:
WHITE: 1156 (77.12%)
OTHER: 151 (10.07%)
BLACK/AFRICAN AMERICAN: 121 (8.07%)
ASIAN: 45 (3.00%)
UNKNOWN/NOT SPECIFIED: 23 (1.53%)
AMERICAN INDIAN/ALASKA NATIVE: 2 (0.13%)
NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER: 1 (0.07%)

Count and Percentage of Each Ethnicity:
Not Hispanic: 1298 (86.59%)
Hispanic: 118 (7.87%)
Unknown: 83 (5.54%)

BDSPPatientIDs, Ages, ContactDates, and DateOfBirths for patients with age > 100 or < 5:


In [30]:
#Total hospital demographics
MGB_Total = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/MGB_total_ICD.csv")
BIDMC_Total = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/BIDMC_total_ICD.csv")

Total = pd.concat([MGB_Total, BIDMC_Total])
Total.to_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/Total_hospitals.csv", index=False)


In [31]:
#Print out demographic information for Both Hospitals
Total = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/Total_hospitals.csv")

#Analyze the 'annot' column. 
total_rows = len(Total)
annot_ones = Total['annot'].eq(1).sum()  
percent_annot_ones = (annot_ones / total_rows) * 100 if total_rows > 0 else 0

#Print count and percentage of rows with 'annot' equal to 1. 
print(f"Count of rows where 'annot' is 1: {annot_ones}")
print(f"Percentage of rows where 'annot' is 1: {percent_annot_ones:.2f}%")

#Extract the 'age' column. 
age = Total['Age'].dropna()  

#Calculate mean age. 
mean_age = age.mean()

#Calculate standard deviation.
std_dev_age = age.std()

#Calculate min and max.
min_age = age.min()
max_age = age.max()

#Calculate IQR (Interquartile Range).
Q1 = age.quantile(0.25)
Q3 = age.quantile(0.75)
IQR = Q3 - Q1

#Print the results.
print(f"Mean Age Both Hospitals: {mean_age}")
print(f"Standard Deviation of Age Both Hospitals: {std_dev_age}")
print(f"Minimum Age Both Hospitals: {min_age}")
print(f"Maximum Age Both Hospitals: {max_age}")
print(f"Interquartile Range (IQR) of Age Both Hospitals: {IQR}")
print(f"Quartile 1 of age Both Hospitals: {Q1}")
print(f"Quartile 3 of age Both Hospitals: {Q3}")


#Analyze 'SexDSC' column.
sex_counts = Total['SexDSC'].value_counts()  
sex_percentages = Total['SexDSC'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Male and Female:")
for sex in sex_counts.index:
    count = sex_counts[sex]
    percent = sex_percentages[sex]
    print(f"{sex}: {count} ({percent:.2f}%)")

#Define the updated mapping for 'PatientRace'
def map_race(race):
    if pd.isna(race):  
        return "UNKNOWN/NOT SPECIFIED"
    race = race.strip()  
    if race in ["WHITE","White"]:
        return "WHITE"
    elif race in ["BLACK/AFRICAN AMERICAN","Black or African American"]:
        return "BLACK/AFRICAN AMERICAN"
    elif race in ["ASIAN","Asian"]:
        return "ASIAN"
    elif race in ["NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER", "Native Hawaiian or Other Pacific Islander"]:
        return "NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER"
    elif race in ["AMERICAN INDIAN/ALASKA NATIVE", "American Indian or Alaska Native"]:
        return "AMERICAN INDIAN/ALASKA NATIVE"
    elif race in ["DECLINED TO ANSWER","Declined"]:
        return "UNKNOWN/NOT SPECIFIED"
    else:
        return "OTHER"


#Apply the mapping to the 'PatientRace' column.
Total['PatientRace'] = Total['PatientRace'].map(map_race)

#Recalculate and print counts and percentages for the modified 'PatientRace' column.
race_counts = Total['PatientRace'].value_counts()  
race_percentages = Total['PatientRace'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Each Race:")
for race in race_counts.index:
    count = race_counts[race]
    percent = race_percentages[race]
    print(f"{race}: {count} ({percent:.2f}%)")

#Define the mapping for 'EthnicGroupDSC'
def map_ethnicity(ethnicity):
    if pd.isna(ethnicity):  
        return "Unknown"
    ethnicity = ethnicity.strip()  
    if ethnicity in ["OTHER ETHNICITY", "OTHER", "PREFER NOT TO SAY", "UNKNOWN/NOT SPECIFIED","UNABLE TO OBTAIN","DECLINED TO ANSWER", 
                     "LATIN AMERICAN", "SOUTH AMERICAN (NOT OTHERWISE SPECIFIED)","Unavailable", "Prefer not to say/Decline"]:
        return "Unknown"
    elif ethnicity in ["SALVADORIAN", "DOMINICAN", "PUERTO RICAN", "GUATEMALAN", "MEXICAN", "COLUMBIAN", "SPANISH", "ARGENTINIAN",
                       "ARGENTINE", "PERUVIAN", "VENEZUELAN", "CHILEAN", "ECUADORIAN", "COSTA RICAN", "CUBAN", "BOLIVIAN", "HONDURAN",
                       "PARAGUAYAN", "NICARAGUAN", "PANAMANIAN", "URUGUAYAN", "EQUATORIAL GUINEAN","MEXICAN, MEXICAN AMERICAN, CHICANO",
                        "SALVADORAN", "COLOMBIAN", "CARIBBEAN ISLAND", "Hispanic" ]:
        return "Hispanic"
    else:
        return "Not Hispanic"
    
#Apply the mapping to the 'EthnicGroupDSC' column.
Total['EthnicGroupDSC'] = Total['EthnicGroupDSC'].map(map_ethnicity)

#Recalculate and print counts and percentages for the modified 'EthnicGroupDSC' column.
ethnicity_counts = Total['EthnicGroupDSC'].value_counts()  
ethnicity_percentages = Total['EthnicGroupDSC'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Each Ethnicity:")
for ethnicity in ethnicity_counts.index:
    count = ethnicity_counts[ethnicity]
    percent = ethnicity_percentages[ethnicity]
    print(f"{ethnicity}: {count} ({percent:.2f}%)")

outliers = Total[(Total['Age'] > 120)]
print("\nBDSPPatientIDs, Ages, ContactDates, and DateOfBirths for patients with age > 100 or < 5:")
for _, row in outliers.iterrows():
    print(f"BDSPPatientID: {row['BDSPPatientID']}, Age: {row['Age']}, ContactDate: {row['ContactDate']}, DateOfBirth: {row['DateOfBirth']}")


Count of rows where 'annot' is 1: 714
Percentage of rows where 'annot' is 1: 23.81%
Mean Age Both Hospitals: 63.345781927309105
Standard Deviation of Age Both Hospitals: 19.715288562835465
Minimum Age Both Hospitals: 0
Maximum Age Both Hospitals: 102
Interquartile Range (IQR) of Age Both Hospitals: 25.0
Quartile 1 of age Both Hospitals: 53.0
Quartile 3 of age Both Hospitals: 78.0

Count and Percentage of Male and Female:
Male: 1619 (53.98%)
Female: 1380 (46.02%)

Count and Percentage of Each Race:
WHITE: 2189 (72.99%)
BLACK/AFRICAN AMERICAN: 297 (9.90%)
OTHER: 294 (9.80%)
ASIAN: 108 (3.60%)
UNKNOWN/NOT SPECIFIED: 104 (3.47%)
AMERICAN INDIAN/ALASKA NATIVE: 5 (0.17%)
NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER: 2 (0.07%)

Count and Percentage of Each Ethnicity:
Not Hispanic: 2400 (80.03%)
Unknown: 429 (14.30%)
Hispanic: 170 (5.67%)

BDSPPatientIDs, Ages, ContactDates, and DateOfBirths for patients with age > 100 or < 5:


In [32]:
#Calculate the demographic information for the ICD + group and ICD - group among both hospitals. 
MGB_Plus = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/mgb_pos_icd.csv")
MGB_Minus = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/mgb_neg_icd.csv")
BIDMC_Plus = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/bidmc_pos_icd.csv")
BIDMC_Minus = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/bidmc_neg_icd.csv")


Plus_total = pd.concat([MGB_Plus, BIDMC_Plus])
Plus_total.to_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/Plus_total_ICD.csv", index=False)

Minus_total=pd.concat([MGB_Minus, BIDMC_Minus])
Minus_total.to_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/Minus_total_ICD.csv", index=False)

In [33]:
#Print out demographic information for Both Hospitals ICD +
Plus = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/Plus_total_ICD.csv")

#Analyze the 'annot' column.
Plus_rows = len(Plus)
annot_ones = Plus['annot'].eq(1).sum()  
percent_annot_ones = (annot_ones / Plus_rows) * 100 if Plus_rows > 0 else 0

#Print count and percentage of rows with 'annot' equal to 1.
print(f"Count of rows where 'annot' is 1: {annot_ones}")
print(f"Percentage of rows where 'annot' is 1: {percent_annot_ones:.2f}%")

#Extract the 'age' column.
age = Plus['Age'].dropna()  

#Calculate mean age
mean_age = age.mean()

#Calculate standard deviation
std_dev_age = age.std()

#Calculate min and max
min_age = age.min()
max_age = age.max()

#Calculate IQR (Interquartile Range)
Q1 = age.quantile(0.25)
Q3 = age.quantile(0.75)
IQR = Q3 - Q1

#Print the results.
print(f"Mean Age Both Hospitals: {mean_age}")
print(f"Standard Deviation of Age Both Hospitals: {std_dev_age}")
print(f"Minimum Age Both Hospitals: {min_age}")
print(f"Maximum Age Both Hospitals: {max_age}")
print(f"Interquartile Range (IQR) of Age Both Hospitals: {IQR}")
print(f"Quartile 1 of age Both Hospitals: {Q1}")
print(f"Quartile 3 of age Both Hospitals: {Q3}")


#Analyze 'SexDSC' column.
sex_counts = Plus['SexDSC'].value_counts()  
sex_percentages = Plus['SexDSC'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Male and Female:")
for sex in sex_counts.index:
    count = sex_counts[sex]
    percent = sex_percentages[sex]
    print(f"{sex}: {count} ({percent:.2f}%)")

#Define the updated mapping for 'PatientRace'.
def map_race(race):
    if pd.isna(race):  
        return "UNKNOWN/NOT SPECIFIED"
    race = race.strip()  
    if race in ["WHITE","White"]:
        return "WHITE"
    elif race in ["BLACK/AFRICAN AMERICAN","Black or African American"]:
        return "BLACK/AFRICAN AMERICAN"
    elif race in ["ASIAN","Asian"]:
        return "ASIAN"
    elif race in ["NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER", "Native Hawaiian or Other Pacific Islander"]:
        return "NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER"
    elif race in ["AMERICAN INDIAN/ALASKA NATIVE", "American Indian or Alaska Native"]:
        return "AMERICAN INDIAN/ALASKA NATIVE"
    elif race in ["DECLINED TO ANSWER","Declined"]:
        return "UNKNOWN/NOT SPECIFIED"
    else:
        return "OTHER"


#Apply the mapping to the 'PatientRace' column.
Plus['PatientRace'] = Plus['PatientRace'].map(map_race)

#Recalculate and print counts and percentages for the modified 'PatientRace' column
race_counts = Plus['PatientRace'].value_counts()  
race_percentages = Plus['PatientRace'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Each Race:")
for race in race_counts.index:
    count = race_counts[race]
    percent = race_percentages[race]
    print(f"{race}: {count} ({percent:.2f}%)")

#Define the mapping for 'EthnicGroupDSC'.
def map_ethnicity(ethnicity):
    if pd.isna(ethnicity):  
        return "Unknown"
    ethnicity = ethnicity.strip()  
    if ethnicity in ["OTHER ETHNICITY", "OTHER", "PREFER NOT TO SAY", "UNKNOWN/NOT SPECIFIED","UNABLE TO OBTAIN","DECLINED TO ANSWER", 
                     "LATIN AMERICAN", "SOUTH AMERICAN (NOT OTHERWISE SPECIFIED)","Unavailable", "Prefer not to say/Decline"]:
        return "Unknown"
    elif ethnicity in ["SALVADORIAN", "DOMINICAN", "PUERTO RICAN", "GUATEMALAN", "MEXICAN", "COLUMBIAN", "SPANISH", "ARGENTINIAN",
                       "ARGENTINE", "PERUVIAN", "VENEZUELAN", "CHILEAN", "ECUADORIAN", "COSTA RICAN", "CUBAN", "BOLIVIAN", "HONDURAN",
                       "PARAGUAYAN", "NICARAGUAN", "PANAMANIAN", "URUGUAYAN", "EQUATORIAL GUINEAN","MEXICAN, MEXICAN AMERICAN, CHICANO",
                        "SALVADORAN", "COLOMBIAN", "CARIBBEAN ISLAND", "Hispanic" ]:
        return "Hispanic"
    else:
        return "Not Hispanic"
    
#Apply the mapping to the 'EthnicGroupDSC' column.
Plus['EthnicGroupDSC'] = Plus['EthnicGroupDSC'].map(map_ethnicity)

#Recalculate and print counts and percentages for the modified 'EthnicGroupDSC' column.
ethnicity_counts = Plus['EthnicGroupDSC'].value_counts()  
ethnicity_percentages = Plus['EthnicGroupDSC'].value_counts(normalize=True) * 100 
print("\nCount and Percentage of Each Ethnicity:")
for ethnicity in ethnicity_counts.index:
    count = ethnicity_counts[ethnicity]
    percent = ethnicity_percentages[ethnicity]
    print(f"{ethnicity}: {count} ({percent:.2f}%)")

outliers = Plus[(Plus['Age'] > 120)]
print("\nBDSPPatientIDs, Ages, ContactDates, and DateOfBirths for patients with age > 100 or < 5:")
for _, row in outliers.iterrows():
    print(f"BDSPPatientID: {row['BDSPPatientID']}, Age: {row['Age']}, ContactDate: {row['ContactDate']}, DateOfBirth: {row['DateOfBirth']}")


Count of rows where 'annot' is 1: 711
Percentage of rows where 'annot' is 1: 47.40%
Mean Age Both Hospitals: 69.598
Standard Deviation of Age Both Hospitals: 16.90053691746899
Minimum Age Both Hospitals: 0
Maximum Age Both Hospitals: 102
Interquartile Range (IQR) of Age Both Hospitals: 21.0
Quartile 1 of age Both Hospitals: 61.0
Quartile 3 of age Both Hospitals: 82.0

Count and Percentage of Male and Female:
Male: 939 (62.60%)
Female: 561 (37.40%)

Count and Percentage of Each Race:
WHITE: 1139 (75.93%)
OTHER: 134 (8.93%)
BLACK/AFRICAN AMERICAN: 126 (8.40%)
ASIAN: 53 (3.53%)
UNKNOWN/NOT SPECIFIED: 45 (3.00%)
AMERICAN INDIAN/ALASKA NATIVE: 2 (0.13%)
NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER: 1 (0.07%)

Count and Percentage of Each Ethnicity:
Not Hispanic: 1257 (83.80%)
Unknown: 164 (10.93%)
Hispanic: 79 (5.27%)

BDSPPatientIDs, Ages, ContactDates, and DateOfBirths for patients with age > 100 or < 5:


In [34]:
#Print out demographic information for Both Hospitals for ICD minus. 
Minus = pd.read_csv("/home/gregory178/Desktop/NAX project/NAX_SDH/Minus_total_ICD.csv")

#Analyze the 'annot' column
Minus_rows = len(Minus)
annot_ones = Minus['annot'].eq(1).sum()  
percent_annot_ones = (annot_ones / Minus_rows) * 100 if Minus_rows > 0 else 0

#Print count and percentage of rows with 'annot' equal to 1.
print(f"Count of rows where 'annot' is 1: {annot_ones}")
print(f"Percentage of rows where 'annot' is 1: {percent_annot_ones:.2f}%")

#Extract the 'age' column.
age = Minus['Age'].dropna()

#Calculate mean age.
mean_age = age.mean()

#Calculate standard deviation.
std_dev_age = age.std()

#Calculate min and max.
min_age = age.min()
max_age = age.max()

#Calculate IQR (Interquartile Range)
Q1 = age.quantile(0.25)
Q3 = age.quantile(0.75)
IQR = Q3 - Q1

#Print the results.
print(f"Mean Age Both Hospitals: {mean_age}")
print(f"Standard Deviation of Age Both Hospitals: {std_dev_age}")
print(f"Minimum Age Both Hospitals: {min_age}")
print(f"Maximum Age Both Hospitals: {max_age}")
print(f"Interquartile Range (IQR) of Age Both Hospitals: {IQR}")
print(f"Quartile 1 of age Both Hospitals: {Q1}")
print(f"Quartile 3 of age Both Hospitals: {Q3}")


#Analyze 'SexDSC' column.
sex_counts = Minus['SexDSC'].value_counts()  
sex_percentages = Minus['SexDSC'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Male and Female:")
for sex in sex_counts.index:
    count = sex_counts[sex]
    percent = sex_percentages[sex]
    print(f"{sex}: {count} ({percent:.2f}%)")

#Define the updated mapping for 'PatientRace'.
def map_race(race):
    if pd.isna(race):  
        return "UNKNOWN/NOT SPECIFIED"
    race = race.strip()  
    if race in ["WHITE","White"]:
        return "WHITE"
    elif race in ["BLACK/AFRICAN AMERICAN","Black or African American"]:
        return "BLACK/AFRICAN AMERICAN"
    elif race in ["ASIAN","Asian"]:
        return "ASIAN"
    elif race in ["NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER", "Native Hawaiian or Other Pacific Islander"]:
        return "NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER"
    elif race in ["AMERICAN INDIAN/ALASKA NATIVE", "American Indian or Alaska Native"]:
        return "AMERICAN INDIAN/ALASKA NATIVE"
    elif race in ["DECLINED TO ANSWER","Declined"]:
        return "UNKNOWN/NOT SPECIFIED"
    else:
        return "OTHER"

#Apply the mapping to the 'PatientRace' column.
Minus['PatientRace'] = Minus['PatientRace'].map(map_race)

#Recalculate and print counts and percentages for the modified 'PatientRace' column.
race_counts = Minus['PatientRace'].value_counts()  
race_percentages = Minus['PatientRace'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Each Race:")
for race in race_counts.index:
    count = race_counts[race]
    percent = race_percentages[race]
    print(f"{race}: {count} ({percent:.2f}%)")

#Define the mapping for 'EthnicGroupDSC'.
def map_ethnicity(ethnicity):
    if pd.isna(ethnicity):  
        return "Unknown"
    ethnicity = ethnicity.strip()  
    if ethnicity in ["OTHER ETHNICITY", "OTHER", "PREFER NOT TO SAY", "UNKNOWN/NOT SPECIFIED","UNABLE TO OBTAIN","DECLINED TO ANSWER", 
                     "LATIN AMERICAN", "SOUTH AMERICAN (NOT OTHERWISE SPECIFIED)","Unavailable", "Prefer not to say/Decline"]:
        return "Unknown"
    elif ethnicity in ["SALVADORIAN", "DOMINICAN", "PUERTO RICAN", "GUATEMALAN", "MEXICAN", "COLUMBIAN", "SPANISH", "ARGENTINIAN",
                       "ARGENTINE", "PERUVIAN", "VENEZUELAN", "CHILEAN", "ECUADORIAN", "COSTA RICAN", "CUBAN", "BOLIVIAN", "HONDURAN",
                       "PARAGUAYAN", "NICARAGUAN", "PANAMANIAN", "URUGUAYAN", "EQUATORIAL GUINEAN","MEXICAN, MEXICAN AMERICAN, CHICANO",
                        "SALVADORAN", "COLOMBIAN", "CARIBBEAN ISLAND", "Hispanic" ]:
        return "Hispanic"
    else:
        return "Not Hispanic"
    
#Apply the mapping to the 'EthnicGroupDSC' column.
Minus['EthnicGroupDSC'] = Minus['EthnicGroupDSC'].map(map_ethnicity)

#Recalculate and print counts and percentages for the modified 'EthnicGroupDSC' column.
ethnicity_counts = Minus['EthnicGroupDSC'].value_counts()  
ethnicity_percentages = Minus['EthnicGroupDSC'].value_counts(normalize=True) * 100  
print("\nCount and Percentage of Each Ethnicity:")
for ethnicity in ethnicity_counts.index:
    count = ethnicity_counts[ethnicity]
    percent = ethnicity_percentages[ethnicity]
    print(f"{ethnicity}: {count} ({percent:.2f}%)")
outliers = Minus[(Minus['Age'] > 120)]
print("\nBDSPPatientIDs, Ages, ContactDates, and DateOfBirths for patients with age > 100 or < 5:")
for _, row in outliers.iterrows():
    print(f"BDSPPatientID: {row['BDSPPatientID']}, Age: {row['Age']}, ContactDate: {row['ContactDate']}, DateOfBirth: {row['DateOfBirth']}")


Count of rows where 'annot' is 1: 3
Percentage of rows where 'annot' is 1: 0.20%
Mean Age Both Hospitals: 57.08939292861908
Standard Deviation of Age Both Hospitals: 20.341444886531676
Minimum Age Both Hospitals: 0
Maximum Age Both Hospitals: 101
Interquartile Range (IQR) of Age Both Hospitals: 28.0
Quartile 1 of age Both Hospitals: 44.0
Quartile 3 of age Both Hospitals: 72.0

Count and Percentage of Male and Female:
Female: 819 (54.64%)
Male: 680 (45.36%)

Count and Percentage of Each Race:
WHITE: 1050 (70.05%)
BLACK/AFRICAN AMERICAN: 171 (11.41%)
OTHER: 160 (10.67%)
UNKNOWN/NOT SPECIFIED: 59 (3.94%)
ASIAN: 55 (3.67%)
AMERICAN INDIAN/ALASKA NATIVE: 3 (0.20%)
NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER: 1 (0.07%)

Count and Percentage of Each Ethnicity:
Not Hispanic: 1143 (76.25%)
Unknown: 265 (17.68%)
Hispanic: 91 (6.07%)

BDSPPatientIDs, Ages, ContactDates, and DateOfBirths for patients with age > 100 or < 5:
